# Apache Spark: Query data as an engineer

In [ ]:
#Download the correct jars
!mkdir -p jars
!cd jars && curl -L -O https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.4.1/iceberg-spark-runtime-3.5_2.12-1.4.1.jar
!cd jars && curl -L -O https://repo1.maven.org/maven2/software/amazon/awssdk/bundle/2.20.160/bundle-2.20.160.jar
!cd jars && curl -L -O https://repo1.maven.org/maven2/software/amazon/awssdk/url-connection-client/2.20.160/url-connection-client-2.20.160.jar

In [ ]:
#!java -version

In [ ]:
#!ls -l /usr/local/opt/openjdk*

In [ ]:
#!ls -l /opt/homebrew/opt/openjdk*

In [ ]:
# Install correct version of jdk
!brew install openjdk@11

In [1]:
#Confirm spark version
import pyspark
print(pyspark.__version__)

3.5.0


In [ ]:
#!ls -l /opt/homebrew/opt/openjdk@11/bin/java

In [1]:
#confirm versions are referenced correctly
import os

# Set these before creating SparkSession
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ.get('PATH', '')}"

# Verify the Java version that will be used
!$JAVA_HOME/bin/java -version

openjdk version "11.0.25" 2024-10-15
OpenJDK Runtime Environment Homebrew (build 11.0.25+0)
OpenJDK 64-Bit Server VM Homebrew (build 11.0.25+0, mixed mode)


In [2]:
from dotenv import load_dotenv
# import pyspark
from pyspark.sql import SparkSession

load_dotenv()

POLARIS_ENGINEER_CLIENT_ID = os.getenv('POLARIS_ENGINEER_CLIENT_ID')
POLARIS_ENGINEER_CLIENT_SECRET = os.getenv('POLARIS_ENGINEER_CLIENT_SECRET')
WAREHOUSE = 'snowflake_catalog'
PRINCIPAL_ENGINEER_ROLE = 'spark_engineer_role'

In [ ]:
#from claud
try:
    spark = SparkSession.builder \
        .appName("polaris-connection") \
        .master("local[1]") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.sql.shuffle.partitions", "1") \
        .config("spark.jars", jars_paths) \
        .config("spark.driver.extraClassPath", jars_paths) \
        .config("spark.executor.extraClassPath", jars_paths) \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.polaris.catalog-impl", "org.apache.iceberg.rest.RESTCatalog") \
        .config("spark.sql.catalog.polaris.uri", "https://sfsenorthamerica-fh_demo_polaris.snowflakecomputing.com/polaris/api/catalog") \
        .config("spark.sql.catalog.polaris.warehouse", WAREHOUSE) \
        .config("spark.sql.catalog.polaris.credential", f"{POLARIS_ENGINEER_CLIENT_ID}:{POLARIS_ENGINEER_CLIENT_SECRET}") \
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", "vended-credentials") \
        .config('spark.sql.catalog.polaris.scope', f"PRINCIPAL_ROLE:{PRINCIPAL_ENGINEER_ROLE}") \
        .config('spark.sql.catalog.polaris.client.region','us-east-1') \
        .config('spark.sql.iceberg.vectorization.enabled', 'false') \
        .getOrCreate()

    print("Successfully created Spark session")
    print("\nJava Home:", os.environ.get('JAVA_HOME'))
    print("Spark Version:", spark.version)
    
    # Test the connection by listing namespaces first
    print("\nListing available namespaces:")
    namespaces = spark.sql("SHOW NAMESPACES IN polaris").show()
    
    # After seeing the namespaces, we can try to list tables in a specific namespace
    # print("\nListing tables in specific namespace:")
    # tables = spark.sql("SHOW TABLES IN polaris.your_namespace").show()

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nDebug Information:")
    print(f"Java Home: {os.environ.get('JAVA_HOME')}")
    print(f"Found JAR files: {jars}")

In [3]:
import os
from pyspark.sql import SparkSession

# Set Java environment
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ.get('PATH', '')}"
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

# Stop any existing SparkSession
if 'spark' in locals():
    spark.stop()

# Your credentials
POLARIS_ENGINEER_CLIENT_ID = 'EiC20Q1tZ9nw+8N1PmTD6MUQwKI='
POLARIS_ENGINEER_CLIENT_SECRET = 'cxZaToxX6KpdT+hLxAgnToKCh6uwmdXnvA8d7XbI/UI='
WAREHOUSE = 'DEMO'
PRINCIPAL_ENGINEER_ROLE = 'spark_engineer_role'


# Get the absolute path to the jars directory
current_dir = os.getcwd()
jars_dir = os.path.join(current_dir, "jars")
jars = [os.path.join(jars_dir, f) for f in os.listdir(jars_dir) if f.endswith('.jar')]
jars_paths = ",".join(jars)



In [3]:
try:
    spark = SparkSession.builder.appName('iceberg') \
        .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.1,software.amazon.awssdk:bundle:2.20.160,software.amazon.awssdk:url-connection-client:2.20.160') \
        .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
        .config('spark.sql.defaultCatalog', 'polaris') \
        .config('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog') \
        .config('spark.sql.iceberg.vectorization.enabled', 'false') \
        .config('spark.sql.catalog.polaris.type', 'rest') \
        .config('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'vended-credentials') \
        .config("spark.sql.catalog.polaris.uri", "https://sfsenorthamerica-fh_demo_polaris.snowflakecomputing.com/polaris/api/catalog") \
        .config('spark.sql.catalog.polaris.credential', f"{POLARIS_ENGINEER_CLIENT_ID}:{POLARIS_ENGINEER_CLIENT_SECRET}") \
        .config('spark.sql.catalog.polaris.warehouse', f"{WAREHOUSE}") \
        .config('spark.sql.catalog.polaris.scope', f"PRINCIPAL_ROLE:{PRINCIPAL_ENGINEER_ROLE}") \
        .config('spark.sql.catalog.polaris.client.region','us-east-1') \
        .getOrCreate()
except Exception as e:
      print(e)

:: loading settings :: url = jar:file:/Users/fhameed/anaconda3/envs/iceberg/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/fhameed/.ivy2/cache
The jars for the packages stored in: /Users/fhameed/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c457f446-2baf-4734-ba15-485b563b5ddd;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.1 in central
	found software.amazon.awssdk#bundle;2.20.160 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.20.160 in central
	found software.amazon.awssdk#utils;2.20.160 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found software.amazon.awssdk#annotations;2.20.160 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.20.160 in central
	found soft

In [4]:
# First, show tables in the RAW schema
print("\nListing tables in OPEN_CATALOG_DEMO.RAW:")
spark.sql("SHOW TABLES IN polaris.OPEN_CATALOG_DEMO.RAW").show()


Listing tables in OPEN_CATALOG_DEMO.RAW:


Py4JJavaError: An error occurred while calling o51.sql.
: org.apache.iceberg.exceptions.BadRequestException: Malformed request: unauthorized_client: The client is not authorized
	at org.apache.iceberg.rest.ErrorHandlers$OAuthErrorHandler.accept(ErrorHandlers.java:199)
	at org.apache.iceberg.rest.ErrorHandlers$OAuthErrorHandler.accept(ErrorHandlers.java:173)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.postForm(HTTPClient.java:378)
	at org.apache.iceberg.rest.auth.OAuth2Util.fetchToken(OAuth2Util.java:191)
	at org.apache.iceberg.rest.RESTSessionCatalog.initialize(RESTSessionCatalog.java:167)
	at org.apache.iceberg.rest.RESTCatalog.initialize(RESTCatalog.java:72)
	at org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:239)
	at org.apache.iceberg.CatalogUtil.buildIcebergCatalog(Cat

In [6]:
spark.sql("SELECT * FROM polaris.OPEN_CATALOG_DEMO.RAW.VEHICLE_INFO").show(truncate=False)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
[Stage 0:>                                                          (0 + 1) / 1]

+----------+-----------+---------+-----+
|VEHICLE_ID|MODEL      |MAKE_YEAR|COLOR|
+----------+-----------+---------+-----+
|V236106   |Sedan      |2021     |Blue |
|V65234    |Convertible|2022     |Green|
|V399909   |Hatchback  |2022     |Green|
|V442698   |SUV        |2018     |White|
|V239605   |Sedan      |2021     |Blue |
|V94538    |Hatchback  |2019     |White|
|V323886   |SUV        |2016     |Red  |
|V249750   |Sedan      |2016     |Blue |
|V293441   |Sedan      |2019     |Black|
|V228508   |Hatchback  |2021     |Red  |
|V13874    |Truck      |2018     |Blue |
|V94224    |Hatchback  |2017     |Blue |
|V325442   |Truck      |2017     |White|
|V84642    |SUV        |2021     |Black|
|V258701   |Sedan      |2016     |Green|
|V210922   |Convertible|2021     |Blue |
|V69074    |Hatchback  |2021     |Blue |
|V101999   |Hatchback  |2015     |Red  |
|V100502   |Hatchback  |2020     |White|
|V463546   |Sedan      |2021     |White|
+----------+-----------+---------+-----+
only showing top

In [5]:
spark.sql("""
INSERT INTO polaris.OPEN_CATALOG_DEMO.RAW.VEHICLE_INFO
VALUES 
    ('V001', 'Camry', 2020, 'Blue'),
    ('V002', 'Accord', 2021, 'Black'),
    ('V003', 'Model 3', 2022, 'Red'),
    ('V004', 'Highlander', 2021, 'Silver'),
    ('V005', 'CRV', 2019, 'White')
""")

24/12/14 01:09:37 ERROR Utils: Aborting task
software.amazon.awssdk.services.s3.model.S3Exception: User: arn:aws:sts::484577546576:assumed-role/fh-open-catalog-access-role/snowflake is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::fh-east1-s3/Snowflake_Open_Catalog/raw/vehicle_info/data/00003-4-dc0f7c08-34cc-43ee-897d-892044024473-00001.parquet" because no session policy allows the s3:PutObject action (Service: S3, Status Code: 403, Request ID: XXJXFGF282K6C88Y, Extended Request ID: GejI8umAEbRJKLtztSLUstl9PGJ4KPMqvIkQ3JMRkeM4yaUuoCJRiyB+JOGLYLhGmW66Jm1tJus=)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleErrorResponse(AwsXmlPredicatedResponseHandler.java:156)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleResponse(AwsXmlPredicatedResponseHandler.java:108)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handle(AwsXmlPr

Py4JJavaError: An error occurred while calling o51.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 1.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1.0 (TID 5) (localhost executor driver): software.amazon.awssdk.services.s3.model.S3Exception: User: arn:aws:sts::484577546576:assumed-role/fh-open-catalog-access-role/snowflake is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::fh-east1-s3/Snowflake_Open_Catalog/raw/vehicle_info/data/00004-5-dc0f7c08-34cc-43ee-897d-892044024473-00001.parquet" because no session policy allows the s3:PutObject action (Service: S3, Status Code: 403, Request ID: XXJXT2E8M6KH9YCN, Extended Request ID: rzLWXz5UHT1bHAhlORfj1ur0tuMLflHL6b6uiLCQv3FfCgsZ8XOuyge/vwBfWJ8kxKUY+bO+/8A=)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleErrorResponse(AwsXmlPredicatedResponseHandler.java:156)
	at software.amazon.awssdk.protocols.xml.internal.unmarsh

24/12/14 01:09:39 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (localhost executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 4 in stage 1.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1.0 (TID 5) (localhost executor driver): software.amazon.awssdk.services.s3.model.S3Exception: User: arn:aws:sts::484577546576:assumed-role/fh-open-catalog-access-role/snowflake is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::fh-east1-s3/Snowflake_Open_Catalog/raw/vehicle_info/data/00004-5-dc0f7c08-34cc-43ee-897d-892044024473-00001.parquet" because no session policy allows the s3:PutObject action (Service: S3, Status Code: 403, Request ID: XXJXT2E8M6KH9YCN, Extended Request ID: rzLWXz5UHT1bHAhlORfj1ur0tuMLflHL6b6uiLCQv3FfCgsZ8XOuyge/vwBfWJ8kxKUY+bO+/8A=)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleErrorResponse(AwsXmlPredicatedResponseHandler.java:156)
	at software.ama